In [21]:
import torch
import KoBart
import dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data file path
dev_path = '/home/josh/Desktop/school/data/dev.tsv'

# config
batch_size = 16

# model, tokenizer init
model = KoBart.KoBARTConditionalGeneration().to(device)
model.load_state_dict(torch.load('/home/josh/Desktop/school/models/kobart/kobart_step_18000.pth'))
tokenizer = model.tokenizer

# dataset
dev_dataset = dataset.KoBARTQGDataset(dev_path, tokenizer)
dev_dataloader = DataLoader(dev_dataset, batch_size)


with open('/home/josh/Desktop/school/outputs/output_18000.txt', 'w', encoding='utf-8') as f:
    for step_index, batch_data in tqdm( enumerate(dev_dataloader), f"[GENERATE]", total=len(dev_dataloader)):

        input_ids, decoder_input_ids, labels = tuple(value.to(device) for value in batch_data.values())

        output = model.model.generate(input_ids=input_ids, eos_token_id=tokenizer.eos_token_id, max_length=100, num_beams=5)

        for o in output:
            output = tokenizer.decode(o, skip_special_tokens=True)
            f.write(output+'\n')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
[GENERATE]: 100%|██████████| 361/361 [01:59<00:00,  3.03it/s]


# Check BLEU SCORES

In [5]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
import nltk
import pandas as pd
import numpy as np
from konlpy.tag import Mecab

# import language_evaluation
from typing import List
from collections import defaultdict, Counter
import re
import math
import sys
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained('Sehong/kobart-QuestionGeneration')

#import mecab_ko_dic
#mecab = Mecab(mecab_ko_dic.MECAB_ARGS)

def mean(lst):
    return sum(lst) / len(lst)


def _calc_ngram_dict(tokens: List[str], ngram: int, dict_ref=None):
    ngram_dict = defaultdict(int) if dict_ref is None else dict_ref
    total = len(tokens)
    for i in range(0, total - ngram + 1):
        item = tuple(tokens[i:i + ngram])
        ngram_dict[item] += 1
    return ngram_dict


def _calc_cover(cand, gold, ngram):
    cand_dict = _calc_ngram_dict(cand, ngram)
    gold_dict = _calc_ngram_dict(gold, ngram)
    cover = 0
    total = 0
    for token, freq in cand_dict.items():
        if token in gold_dict:
            cover += min(freq, gold_dict[token])
        total += freq
    return cover, total


def _calc_cover_rate(cands, golds, ngram):
    """
    calc_cover_rate
    """
    cover = 0.0
    total = 0.000001
    for cand_tokens, gold_tokens in zip(cands, golds):
        cur_cover, cur_total = _calc_cover(cand_tokens, gold_tokens, ngram)
        cover += cur_cover
        total += cur_total
    return cover / total


def _calc_bp(cands, golds):
    c_count = 0.000001
    r_count = 0.0
    for cand_tokens, gold_tokens in zip(cands, golds):
        c_count += len(cand_tokens)
        r_count += len(gold_tokens)
    bp = 1
    if c_count < r_count:
        bp = math.exp(1 - r_count / c_count)
    return bp


def calc_corpus_bleu(cands, golds):
    bp = _calc_bp(cands, golds)
    cover_rate1 = _calc_cover_rate(cands, golds, 1)
    cover_rate2 = _calc_cover_rate(cands, golds, 2)
    cover_rate3 = _calc_cover_rate(cands, golds, 3)
    bleu1 = 0
    bleu2 = 0
    bleu3 = 0
    if cover_rate1 > 0:
        bleu1 = bp * math.exp(math.log(cover_rate1))
    if cover_rate2 > 0:
        bleu2 = bp * math.exp((math.log(cover_rate1) + math.log(cover_rate2)) / 2)
    if cover_rate3 > 0:
        bleu3 = bp * math.exp((math.log(cover_rate1) + math.log(cover_rate2) + math.log(cover_rate3)) / 3)
    return bleu1, bleu2, bleu3


def calc_sentence_bleu(cands, golds):
    bleu1 = []
    bleu2 = []
    bleu3 = []
    sf = SmoothingFunction().method7
    for hyp, ref in zip(cands, golds):
        try:
            b1 = sentence_bleu([ref], hyp, smoothing_function=sf, weights=[1, 0, 0, 0])
        except ZeroDivisionError:
            b1 = 0.0
        try:
            b2 = sentence_bleu([ref], hyp, smoothing_function=sf, weights=[0.5, 0.5, 0, 0])
        except ZeroDivisionError:
            b2 = 0.0
        try:
            b3 = sentence_bleu([ref], hyp, smoothing_function=sf, weights=[0.34, 0.33, 0.33, 0])
        except ZeroDivisionError:
            b3 = 0.0
        bleu1.append(b1)
        bleu2.append(b2)
        bleu3.append(b3)
    return mean(bleu1), mean(bleu2), mean(bleu3)


def calc_corpus_bleu_new(hypothesis, references):
    # hypothesis = [normalize_answer(hyp).split(" ") for hyp in hypothesis]
    # references = [[normalize_answer(ref).split(" ")] for ref in references]
    references = [[gold] for gold in references]
    sf = SmoothingFunction(epsilon=1e-12).method1
    b1 = corpus_bleu(references, hypothesis, weights=(1.0 / 1.0,), smoothing_function=sf)
    b2 = corpus_bleu(references, hypothesis, weights=(1.0 / 2.0, 1.0 / 2.0), smoothing_function=sf)
    b3 = corpus_bleu(references, hypothesis, weights=(1.0 / 3.0, 1.0 / 3.0, 1.0 / 3.0), smoothing_function=sf)
    b4 = corpus_bleu(references, hypothesis, weights=(1.0 / 4.0, 1.0 / 4.0, 1.0 / 4.0, 1.0 / 4.0),
                     smoothing_function=sf)
    return b1, b2, b3, b4


def _calc_distinct_ngram(cands, ngram):
    ngram_total = 0.00001
    ngram_distinct_count = 0.00001
    pred_dict = defaultdict(int)
    for cand_tokens in cands:
        _calc_ngram_dict(cand_tokens, ngram, pred_dict)
    for key, freq in pred_dict.items():
        ngram_total += freq
        ngram_distinct_count += 1
    return ngram_distinct_count / ngram_total


def _calc_sent_distinct_ngram(cand, ngram):
    ngram_total = 0.0000000001
    ngram_distinct_count = 0.0
    ngram_dict = defaultdict(int)
    for i in range(0, len(cand) - ngram + 1):
        item = tuple(cand[i:i + ngram])
        ngram_dict[item] += 1
    for _, freq in ngram_dict.items():
        ngram_total += freq
        ngram_distinct_count += 1
    return ngram_distinct_count / ngram_total


def calc_corpus_distinct(cands):
    distinct1 = _calc_distinct_ngram(cands, 1)
    distinct2 = _calc_distinct_ngram(cands, 2)
    return distinct1, distinct2


def calc_sentence_distinct(cands):
    distinct1 = mean([_calc_sent_distinct_ngram(c, 1) for c in cands])
    distinct2 = mean([_calc_sent_distinct_ngram(c, 2) for c in cands])
    return distinct1, distinct2


def calc_corpus_f1(cands, golds):
    golden_word_total = 0.00000001
    pred_word_total = 0.00000001
    hit_word_total = 0.00000001
    for response, golden_response in zip(cands, golds):
        common = Counter(response) & Counter(golden_response)
        hit_word_total += sum(common.values())
        golden_word_total += len(golden_response)
        pred_word_total += len(response)
    p = hit_word_total / pred_word_total
    r = hit_word_total / golden_word_total
    f1 = 2 * p * r / (p + r)
    return f1


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    re_art = re.compile(r'\b(a|an|the)\b')
    re_punc = re.compile(r'[!"#$%&()*+,-./:;<=>?@\[\]\\^`{|}~_\']')

    def remove_articles(text):
        return re_art.sub(' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        return re_punc.sub(' ', text)  # convert punctuation to spaces

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s)))).split(' ')


def dialogue_evaluation(ori_cands, ori_golds):
    assert len(ori_cands) == len(ori_golds), f"num cand: {len(ori_cands)}, num gold: {len(ori_golds)}"
    cands = []
    golds = []
    
    # 둘 중 원하는 토크나이저로 성능 평가하세요
    
    #help_tokenize = lambda x: mecab.morphs(x.lower())         # Mecab
    help_tokenize = lambda x: tokenizer.encode(x.lower())     # 학습에 사용한 토크나이저
    
    for cand, gold in zip(ori_cands, ori_golds):
        cands.append(help_tokenize(str(cand).lower()))
        golds.append(help_tokenize(str(gold).lower()))
    cbleu1, cbleu2, cbleu3, cbleu4 = calc_corpus_bleu_new(cands, golds)
    sbleu1, sbleu2, sbleu3 = calc_sentence_bleu(cands, golds)
    cdiv1, cdiv2 = calc_corpus_distinct(cands)
    sdiv1, sdiv2 = calc_sentence_distinct(cands)
    cf1 = calc_corpus_f1(cands, golds)
    # rouge_result = calc_rouge(cands, golds)
    result = {
        'cf1': cf1,
        'bleu1': cbleu1,
        'bleu2': cbleu2,
        'bleu3': cbleu3,
        'bleu4': cbleu4,
        'dist1': cdiv1,
        'dist2': cdiv2,
    }
    # result.update(rouge_result)
    result = {k: round(100 * v, 6) for k, v in result.items()}
    return result


def file_dialogue_evaluation(cand_file, gold_file):
    print(f"cand file: {cand_file}, gold file: {gold_file}")
    cands = []
    golds = []
    with open(cand_file, 'r', encoding='utf-8') as f:
        for line in f:
            cands.append(line.strip())
    with open(gold_file, 'r', encoding='utf-8') as f:
        for line in f:
            golds.append(line.strip())
    results = dialogue_evaluation(cands, golds)
    print(results)

# BLEU

In [23]:
test_data_path = '/home/josh/Desktop/school/data/dev.tsv' # .tsv
output_path = '/home/josh/Desktop/school/outputs/output_18000.txt' # .txt

In [25]:
test_data_path = '/home/josh/Desktop/school/data/dev.tsv' # .tsv
output_path = '/home/josh/Desktop/school/outputs/output_18000.txt' # .txt

from tqdm import tqdm
#df = pd.read_csv(test_data_path, encoding='cp949', sep='\t')
df = pd.read_csv(test_data_path, encoding='utf-8', sep='\t')
question = list(df['question'])

generated_question = []


with open(output_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        print(line)
        generated_question.append(line)

# save info
pd.DataFrame(generated_question, question)

len_question = len(question)

score_list_1 = np.zeros(len_question)
score_list_2 = np.zeros(len_question)
score_list_3 = np.zeros(len_question)
score_list_4 = np.zeros(len_question)

for i in tqdm(range(len_question)):
    
    matrix = dialogue_evaluation([generated_question[i]], [question[i]])
    score_list_1[i] = matrix['bleu1']
    score_list_2[i] = matrix['bleu2']
    score_list_3[i] = matrix['bleu3']
    score_list_4[i] = matrix['bleu4']
    

#score_list = np.array(score_list)
print('bleu1: ', np.mean(score_list_1))
print('bleu2: ', np.mean(score_list_2))
print('bleu3: ', np.mean(score_list_3))
print('bleu4: ', np.mean(score_list_4))

서울지방검찰청에서 임종석의 사전구속영장을 발부 받은 날짜는?

서울지방검찰청 공안부가 임종석의 사전구속영장을 발부받은 날짜는?

서울지방검찰청에서 임종석의 사전구속영장을 발부받은 날짜는?

서울지방검찰청 공안부가 임종석의 사전구속영장을 발부받은 날짜는?

서울지방검찰청이 임종석의 사전구속영장을 발부 받은 날짜는?

서울지방검찰청 공안부가 임종석의 사전구속영장을 발부받은 날짜는?

서울지방검찰청이 임종석의 사전구속영장을 발부 받은 날짜는?

정부의 헌법개정안 준비 과정에 대해 청와대가 아닌 국무회의 중심으로 했어야 된다고 지적한 원로 헌법학자는?

헌법 규정에 충실하지 않은 민정수석이 개정안을 설명하는 게 이해가 안 든다고 지적한 사람은?

정부의 헌법개정안 준비 과정에 대해 헌법 규정에 충실하지 않았다고 지적한 사람은?

국무총리 선출 방식에 대한 기자의 질문에 문 대통령은 취임 전까지 총리에게 실질적 권한을 주겠다고 했지만 그전에는 어떤 권한을 주었나?

알렉산더 메이그스 헤이그 2세가 미국 군대에서 2번째로 높은 직위는?

알렉산더 메이그스 헤이그 2세가 미국 군대에서 2번째로 높은 직위는?

알렉산더 메이그스 헤이그 2세의 회고록의 저자는?

알렉산더 메이그스 헤이그 2세가 미국 군대에서 2번째로 높은 직위는?

알렉산더 메이그스 헤이그 2세가 미국 군대에서 2번째로 높은 직위인 미국 육군 부참모 총장은?

알렉산더 메이그스 헤이그 2세가 미국 군대에서 2번째로 높은 직위인 미국 육군 부참모 총장과 미국 군대의 유럽연합군 최고사령관이었던 직위는?

알렉산더 메이그스 헤이그 2세의 회고록의 제목은?

알렉산더 메이그스 헤이그 2세가 미국 군대에서 2번째로 높은 직위를 가진 직위는?

미국 육군사관학교로 임명을 획득한 해는?

미국 육군사관학교로 임명을 획득한 해는?

헤이그는 1947년 310의 동기병에서 몇번째 사관으로서 졸업했는가?

미국 육군사관학교에 임명을 획득한 해는?

헤이그는 몇살에 소위로 기갑 훈련소로 갔는가?

미국 육군사관학교로 임명을 획

100%|██████████| 5774/5774 [00:02<00:00, 2501.51it/s]

bleu1:  14.471179482680984
bleu2:  10.184096950121232
bleu3:  7.191299425874609
bleu4:  4.979940999307239


# Testing original model

In [3]:
import torch
import KoBart
import dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import BartForConditionalGeneration
from transformers import PreTrainedTokenizerFast

# device
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

device = 'cuda'

# Data file path
#'/content/drive/MyDrive/school/CS NLP/'
dev_path = '/home/josh/Desktop/school/data/dev.tsv'

# config
batch_size = 16

# model, tokenizer init
model_directory = 'Sehong/kobart-QuestionGeneration'
model = BartForConditionalGeneration.from_pretrained(model_directory)
tokenizer = PreTrainedTokenizerFast.from_pretrained('Sehong/kobart-QuestionGeneration')

model = model.to(device)

# dataset
dev_dataset = dataset.KoBARTQGDataset(dev_path, tokenizer)
dev_dataloader = DataLoader(dev_dataset, batch_size)

#output_path = './output/.txt'

with open('/home/josh/Desktop/school/outputs/base-Kobart.txt', 'w', encoding='utf-8') as f:
    for step_index, batch_data in tqdm( enumerate(dev_dataloader), f"[GENERATE]", total=len(dev_dataloader)):

        input_ids, decoder_input_ids, labels = tuple(value.to(device) for value in batch_data.values())

        output = model.generate(input_ids=input_ids, eos_token_id=tokenizer.eos_token_id, max_length=100, num_beams=5)

        for o in output:
            output = tokenizer.decode(o, skip_special_tokens=True)
            f.write(output+'\n')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
[GENERATE]: 100%|██████████| 361/361 [02:04<00:00,  2.90it/s]


In [6]:
test_data_path = '/home/josh/Desktop/school/data/dev.tsv' # .tsv
output_path = '/home/josh/Desktop/school/outputs/base-Kobart.txt' # .txt

from tqdm import tqdm
#df = pd.read_csv(test_data_path, encoding='cp949', sep='\t')
df = pd.read_csv(test_data_path, encoding='utf-8', sep='\t')
question = list(df['question'])

generated_question = []


with open(output_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        print(line)
        generated_question.append(line)

# save info
pd.DataFrame(generated_question, question)

len_question = len(question)

score_list_1 = np.zeros(len_question)
score_list_2 = np.zeros(len_question)
score_list_3 = np.zeros(len_question)
score_list_4 = np.zeros(len_question)

for i in tqdm(range(len_question)):
    
    matrix = dialogue_evaluation([generated_question[i]], [question[i]])
    score_list_1[i] = matrix['bleu1']
    score_list_2[i] = matrix['bleu2']
    score_list_3[i] = matrix['bleu3']
    score_list_4[i] = matrix['bleu4']
    

#score_list = np.array(score_list)
print('bleu1: ', np.mean(score_list_1))
print('bleu2: ', np.mean(score_list_2))
print('bleu3: ', np.mean(score_list_3))
print('bleu4: ', np.mean(score_list_4))

임종석이 지명수배된 날짜는?

임종석이 지명수배된 날짜는?

임종석이 지명수배된 날짜는?

임종석이 지명수배된 날짜는?

임종석이 지명수배된 날짜는?

임종석이 지명수배된 날짜는?

임종석이 지명수배된 날짜는?

허영 경희대 석좌교수는 헌법개정안 준비 과정에 대해 "청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 한다고 지적한 것은?

허영 경희대 석좌교수는 헌법개정안 준비 과정에 대해 "청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 한다고 지적한 것은?

허영 경희대 석좌교수는 헌법개정안 준비 과정에 대해 "청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 한다고 지적한 것은?

허영 경희대 석좌교수는 헌법개정안 준비 과정에 대해 "청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 한다고 지적한 것은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

알렉산더 메이그스 헤이그 2세가 쓴 책의 이름은?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

로저 헤이그는 몇 명의 자식을 두었는가?

워터게이트 사건이 일어난지 한달 후 닉슨 대통령이 그를 사성 장군과 육군 부참모로 진급시킨 협상들에서 헤이그가 수단이었던 우연이 아니었던 것은?

워터게이트 사건이 일어난지 한달 후 닉슨 대통령이 그를 사성 장군과 육군 부참모로 진급시킨 협상들에서 헤이그가 수단이었던 우연이 아니었던 것은?

워터게이트 사건이

100%|██████████| 5774/5774 [00:02<00:00, 2319.14it/s]

bleu1:  14.042919400242466
bleu2:  9.935410876515414
bleu3:  7.109965319189469
bleu4:  4.878432230342916


bleu1:  14.471179482680984
bleu2:  10.184096950121232
bleu3:  7.191299425874609
bleu4:  4.979940999307239

In [34]:
data = pd.read_csv(dev_path, sep='\t')

data['generated_question'] = generated_question

data.head()

,content,question,generated_question
0,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?,서울지방검찰청에서 임종석의 사전구속영장을 발부 받은 날짜는?\n
1,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,1989년 6월 30일 평양축전에 대표로 파견 된 인물은?,서울지방검찰청 공안부가 임종석의 사전구속영장을 발부받은 날짜는?\n
2,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?,서울지방검찰청에서 임종석의 사전구속영장을 발부받은 날짜는?\n
3,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,임종석을 검거한 장소는 경희대 내 어디인가?,서울지방검찰청 공안부가 임종석의 사전구속영장을 발부받은 날짜는?\n
4,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,임종석이 조사를 받은 뒤 인계된 곳은 어딘가?,서울지방검찰청이 임종석의 사전구속영장을 발부 받은 날짜는?\n


In [38]:
data.to_csv("/home/josh/Desktop/school/outputs/human_eval.csv")